In [1]:
#train model on horses
from ultralytics import YOLO

model = YOLO("yolo11n-pose.pt")  # or .yaml for building from scratch
horse_results = model.train(data="data.yml", epochs=100, imgsz=640)

ImportError: libopenh264.so.5: cannot open shared object file: No such file or directory

In [ ]:
import cv2
from ultralytics import YOLO

pose_model = YOLO("yolov8n-pose.pt")
horse_model = YOLO("yolov8n.pt")

cap = cv2.VideoCapture("riding1.mp4")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Detect horse
    horse_preds = horse_model.predict(frame, classes=[18], verbose=False)[0]
    horses = [box.xyxy[0].cpu().numpy() for box in horse_preds.boxes]

    # Detect rider pose
    pose_preds = pose_model.predict(frame, verbose=False)[0]
    riders = [box.xyxy[0].cpu().numpy() for box in pose_preds.boxes]

    # Match: simple IOU or centroid distance
    for rider in riders:
        x1_r, y1_r, x2_r, y2_r = rider
        rider_center = ((x1_r + x2_r)/2, (y1_r + y2_r)/2)

        for horse in horses:
            x1_h, y1_h, x2_h, y2_h = horse
            horse_center = ((x1_h + x2_h)/2, (y1_h + y2_h)/2)

            dx = rider_center[0] - horse_center[0]
            dy = rider_center[1] - horse_center[1]
            distance = (dx**2 + dy**2) ** 0.5

            if distance < 100:  # threshold, tune based on your footage
                print("Matched rider to horse at distance", distance)
                # Optional: draw a line
                #cv2.line(frame, rider_center, horse_center, (0, 255, 0), 2)

    cv2.imshow("frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

